# Calculating Multi-Modal Concordance

In [1]:
from azureml.core import Workspace
from model_drift import settings
from model_drift.azure_utils import download_metrics_file
from model_drift.drift.io import load_metrics_file, load_weights, load_stats
from model_drift.drift.unify import calculate_mmc
from model_drift.helpers import filter_columns

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azure-mgmt-resource 13.0.0 (d:\code\mlopsday2\.venv\lib\site-packages), Requirement.parse('azure-mgmt-resource<21.0.0,>=15.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azure-mgmt-resource 13.0.0 (d:\code\mlopsday2\.venv\lib\site-packages), Requirement.parse('azure-mgmt-resource<21.0.0,>=15.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azure-mgmt-resource 13.0.0 (d:\code\mlopsday2\.venv\lib\site-packages), Requirement.parse('azure-mgmt-resource<21.0.0,>=15.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml

In [2]:
# Get results file from azure ml
workspace = Workspace.from_config(settings.AZUREML_CONFIG)
experiment = "generate-drift-metrics"
run = "good_stamp_bdz1lskm"
results_csv = download_metrics_file(run, settings.RESULTS_DIR.joinpath('drift'), experiment, workspace, overwrite=False)



In [3]:
performance_col = ("performance", "micro avg", "auroc")
weights_file = settings.MODEL_DIR.joinpath("weights", "metric_weights.csv")
stats_file = settings.MODEL_DIR.joinpath("weights", "std_stats.csv")


include_metrics = ["distance"]

weights = load_weights(weights_file)
std_stats = load_stats(stats_file)
error_df, results_df = load_metrics_file(results_csv)

perf_df = results_df[performance_col]
error_perf_df = error_df[performance_col]
metrics_df = filter_columns(results_df, exclude=['performance', 'count'])
metrics_df = filter_columns(metrics_df, include=include_metrics)

mmc = calculate_mmc(metrics_df, weights=weights, std_stats=std_stats)


mmc.head()

2012-11-06   -2.520399
2012-11-07   -1.824211
2012-11-08   -1.680284
2012-11-09   -1.503582
2012-11-10   -1.496547
dtype: float64

In [4]:
from model_drift.figure_helper import FigureHelper
import pandas as pd

def smooth(y: pd.DataFrame, span=7):
    if span > 0:
        ys = y.ewm(span=span, ignore_na=False).mean()
        ys[y.isna()] = None
    else:
        ys = y    
    return ys

graph_start = "2014-04-01"
graph_end = "2015-02-01"

x = pd.date_range(pd.to_datetime(graph_start)-pd.DateOffset(n=30), pd.to_datetime(graph_end)+pd.DateOffset(n=30))
fh = FigureHelper(x, color_list=['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#8C564B', '#7f7f7f'])

perf_df = perf_df.reindex(x)
error_perf_df = error_perf_df.reindex(x)
mmc = mmc.reindex(x)

fh.add_trace(y=smooth(perf_df, span=7), 
                 yu=smooth(perf_df+error_perf_df, span=7),
                 yl=smooth(perf_df-error_perf_df, span=7), name=run, connectgaps=False, row=1, col=1, showlegend=True)

fh.add_trace(y=smooth(mmc, span=7), name=run, connectgaps=False, row=2, col=1)

fig = fh.make_fig(shared_xaxes=True, vertical_spacing=0.025)
fig.update_xaxes(showspikes=True, spikecolor="black", spikesnap="cursor", spikemode="across", spikethickness=1, 
                 range=[graph_start, graph_end])
fig.update_layout(spikedistance=1000, height=500, plot_bgcolor="#E8E8EA", title="Multi-Modal Data Concordance")

xaxis = dict(
        tickformat = '%Y-%m-%d',
        tickmode = 'linear',
        dtick = "M1"
    )
fig.update_layout(
    xaxis1=xaxis,
    xaxis2=xaxis,
)

fig.show()